In [1]:
import pandas as pd

df = pd.read_csv('file/health_result.csv')
df.head()

,question,answer,category02
0,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.,29.대기실 및 진료실 위치 안내
1,교정기가 떨어진 것 같아서 붙이러 왔어요.,접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.,27.진료 접수 안내
2,앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?,예약하실 때 지난 번 진료 기록이 있다고 말씀해 주시길 부탁 드립니다. 감사합니다.,33.다음 진료실 예약
3,이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.,이곳에서도 수납 진행 가능합니다. 홈버튼을 누른 후 수납 버튼을 누르십시오.,31.수납 방법 안내
4,주사실 안에 가서 있으라는데 주사실이 어딘데요?,침을 맞는 곳이라 이름이 주사실입니다. 3층으로 올라가시면 됩니다.,29.대기실 및 진료실 위치 안내


In [2]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # KcBERT 모델 로드 (한국어 모욕성 표현 감지)
# tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
# model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base")

# def detect_offensive(sentence):
#     # 디바이스 설정 (GPU 사용 가능 여부 확인)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # 모델을 GPU로 이동
#     model.to(device)
#     inputs = tokenizer(sentence, return_tensors="pt").to(device)
#     outputs = model(**inputs)
#     prediction = outputs.logits.argmax(dim=-1)
#     return prediction.item()  # 0: 정상 문장, 1: 모욕적 문장

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()
# # 질문들의 임베딩 벡터 미리 계산
# def compute_question_embeddings(questions):
#     embeddings = []
#     for question in questions:
#         inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         # 문장 전체의 임베딩을 평균으로 취함
#         embedding = outputs.last_hidden_state.mean(dim=1)
#         embeddings.append(embedding)
#     return embeddings

# # 질문에 대한 임베딩 계산
# question_embeddings = compute_question_embeddings(questions)

# # 임베딩을 저장할 때 torch.save() 사용 가능
# torch.save(question_embeddings, 'question_embeddings.pth')


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch.nn.functional as F

question_embeddings = torch.load('question_embeddings.pth', weights_only=True)

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 입력 질문 예시
input_question = "정수기 위치가 어디죠?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)


챗봇 답변: 위치 정보를 조회하겠습니다. 병원에서 50m 떨어져 있는 곳에 위치해 있습니다.
유사도: 0.8512148857116699


In [6]:
df['category02'].unique()

array(['29.대기실 및 진료실 위치 안내', '27.진료 접수 안내', '33.다음 진료실 예약', '31.수납 방법 안내',
       '30.약국 위치 문의', '28.증상 상담', '32.증빙 서류 발급', '33.증빙 서류 발급',
       '27.증빙 서류 발급', '27.진료 접수', '32.다음 진료실 예약', '34.다음 진료실 예약'],
      dtype=object)

In [7]:
df['category02'].value_counts()

category02
29.대기실 및 진료실 위치 안내    4994
30.약국 위치 문의           4933
28.증상 상담              4921
32.증빙 서류 발급           4905
31.수납 방법 안내           4903
27.진료 접수 안내           4898
33.다음 진료실 예약          4884
32.다음 진료실 예약             3
33.증빙 서류 발급              2
27.진료 접수                 2
27.증빙 서류 발급              1
34.다음 진료실 예약             1
Name: count, dtype: int64

In [8]:
# 고유 카테고리 추출
unique_categories = df['category02'].unique()

# 카테고리와 고유 번호 매핑
category_to_label = {category: idx for idx, category in enumerate(unique_categories)}

# 레이블 추가
df['label'] = df['category02'].map(category_to_label)

# 결과 확인
df.head()
df['label'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split
from collections import Counter
import random
# CUDA 디버깅을 위한 환경 변수 설정
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# 질문과 답변을 결합
df['input'] = df['question'] + " [SEP] " + df['answer']  # [SEP]는 BERT에서 문장을 구분하는 토큰입니다.

# 훈련 데이터와 평가 데이터로 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    df['input'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# 모든 클래스 정의 (예: 0부터 11까지)
all_classes = set(range(12))  # 모델의 클래스 수에 따라 조정하세요

# 훈련 데이터에서 클래스 확인
train_class_counts = Counter(train_labels)
eval_class_counts = Counter(eval_labels)
missing_classes = [cls for cls in all_classes if cls not in train_class_counts]
missing_classes2 = [cls for cls in all_classes if cls not in eval_class_counts]

# 부족한 클래스 샘플 보충 (원본 데이터프레임에서 무작위로 선택)
for cls in missing_classes:
    # 해당 클래스의 인덱스 찾기
    cls_indices = [i for i, label in enumerate(df['label']) if label == cls]
    if cls_indices:  # 해당 클래스에 샘플이 존재하는 경우
        # 원본 데이터에서 하나의 샘플을 무작위로 선택
        sample_index = random.choice(cls_indices)
        train_texts.append(df['input'][sample_index])  # 훈련 데이터에 샘플 추가
        train_labels.append(cls)  # 레이블 추가

# 부족한 클래스 샘플 보충 (원본 데이터프레임에서 무작위로 선택)
for cls in missing_classes2:
    # 해당 클래스의 인덱스 찾기
    cls_indices = [i for i, label in enumerate(df['label']) if label == cls]
    if cls_indices:  # 해당 클래스에 샘플이 존재하는 경우
        # 원본 데이터에서 하나의 샘플을 무작위로 선택
        sample_index = random.choice(cls_indices)
        eval_texts.append(df['input'][sample_index])  # 훈련 데이터에 샘플 추가
        eval_labels.append(cls)  # 레이블 추가

# 결과 확인
print("Updated train labels:", Counter(train_labels))
print(set(train_labels))
print(set(eval_labels))

Updated train labels: Counter({0: 3983, 3: 3957, 4: 3948, 6: 3927, 1: 3927, 5: 3910, 2: 3899, 10: 3, 7: 2, 8: 1, 9: 1, 11: 1})
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

# 토크나이징
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)


In [11]:
import torch


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, num_classes):
        self.encodings = encodings
        self.labels = labels

        # 레이블 유효성 검사
        assert all(label >= 0 for label in self.labels), "Negative label found!"
        assert all(label < num_classes for label in self.labels), "Label out of range!"

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# 데이터셋 생성
num_classes = 12  # 모델의 클래스 수
train_dataset = CustomDataset(train_encodings, train_labels, num_classes)
eval_dataset = CustomDataset(eval_encodings, eval_labels, num_classes)


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    logging_dir='./logs',
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 모델 훈련
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                   
 54%|█████▎    | 2767/5169 [42:43<11:26,  3.50it/s]

{'loss': 0.2432, 'grad_norm': 5.247402667999268, 'learning_rate': 4.5163474559876186e-05, 'epoch': 0.29}


                                                   
 54%|█████▎    | 2767/5169 [45:17<11:26,  3.50it/s]

{'loss': 0.0607, 'grad_norm': 1.7995136976242065, 'learning_rate': 4.032694911975237e-05, 'epoch': 0.58}


                                                   
 54%|█████▎    | 2767/5169 [47:46<11:26,  3.50it/s]

...

                                                     
 54%|█████▎    | 2767/5169 [1:06:52<11:26,  3.50it/s]

{'loss': 0.0045, 'grad_norm': 0.0013340313453227282, 'learning_rate': 1.6347455987618494e-06, 'epoch': 2.9}















































































































































































































































                                                     

                                           
 54%|█████▎    | 2767/5169 [1:09:04<11:26,  3.50it/s]

                                                     
100%|██████████| 5169/5169 [28:32<00:00,  3.02it/s]s]

{'eval_loss': 0.020677322521805763, 'eval_runtime': 28.5278, 'eval_samples_per_second': 241.624, 'eval_steps_per_second': 30.216, 'epoch': 3.0}
{'train_runtime': 1713.1746, 'train_samples_per_second': 48.26, 'train_steps_per_second': 3.017, 'train_loss': 0.040529435632030056, 'epoch': 3.0}


TrainOutput(global_step=5169, training_loss=0.040529435632030056, metrics={'train_runtime': 1713.1746, 'train_samples_per_second': 48.26, 'train_steps_per_second': 3.017, 'total_flos': 2931851219277960.0, 'train_loss': 0.040529435632030056, 'epoch': 3.0})

In [14]:
# 전체 모델 저장
torch.save(model, 'model/full_model.pth')

In [12]:
# 전체 모델 불러오기
model = torch.load('model/full_model.pth')
model.eval()  # 평가 모드로 전환 (필요한 경우)


<ipython-input-12-2f4d98fedd92>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('full_model.pth')


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
# 모델의 상태 사전 저장 -> 호환성이 더 좋음
torch.save(model.state_dict(), 'model/model_weights.pth')

In [17]:
# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 저장된 상태 사전 불러오기 (weights_only=True 설정)
model.load_state_dict(torch.load('model/model_weights.pth', weights_only=True))

# 평가 모드로 전환
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
trainer.evaluate()

100%|██████████| 862/862 [00:30<00:00, 28.39it/s]


{'eval_loss': 0.020677322521805763,
 'eval_runtime': 30.2996,
 'eval_samples_per_second': 227.495,
 'eval_steps_per_second': 28.449,
 'epoch': 3.0}

In [17]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# 성능 평가 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Accuracy
    accuracy = accuracy_score(labels, predictions)

    # F1 Score (macro average for multi-class classification)
    f1 = f1_score(labels, predictions, average="macro")

    return {
        "accuracy": accuracy,
        "f1": f1,
    }

# Optuna 최적화 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 제안
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    batch_size = trial.suggest_int("batch_size", 8, 32)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs= num_train_epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        save_total_limit=1,
        load_best_model_at_end=True,
    )

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,  # 성능 평가 함수 추가
    )

    # 모델 학습
    trainer.train()

    # 평가 및 최적화 지표 반환
    eval_result = trainer.evaluate()
    return eval_result["eval_loss"]

# Optuna 최적화 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# 최적화된 하이퍼파라미터 출력
print(f"Best hyperparameters: {study.best_trial.params}")


[I 2024-10-09 06:37:14,473] A new study created in memory with name: no-name-17bdfa0f-2063-4d51-b991-606e779bb87e
                                                     
 54%|█████▎    | 2767/5169 [1:10:02<11:26,  3.50it/s]

{'loss': 0.0002, 'grad_norm': 0.0011158266570419073, 'learning_rate': 1.0674661083923695e-05, 'epoch': 0.01}


                                                     
 54%|█████▎    | 2767/5169 [1:10:05<11:26,  3.50it/s]

{'loss': 0.0001, 'grad_norm': 0.0013438223395496607, 'learning_rate': 1.065332029807019e-05, 'epoch': 0.02}


                                                     
 54%|█████▎    | 2767/5169 [1:10:08<11:26,  3.50it/s]

...

{'eval_loss': 0.04957032948732376, 'eval_accuracy': 0.9940519367474249, 'eval_f1': 0.5801321330950082, 'eval_runtime': 34.5825, 'eval_samples_per_second': 199.32, 'eval_steps_per_second': 24.926, 'epoch': 3.0}


                                                     
100%|██████████| 4866/4866 [27:39<00:00,  2.93it/s]s]


{'train_runtime': 1659.0947, 'train_samples_per_second': 49.833, 'train_steps_per_second': 2.933, 'train_loss': 0.025881251400963246, 'epoch': 3.0}


100%|██████████| 862/862 [00:34<00:00, 24.91it/s]
[I 2024-10-09 11:50:45,190] Trial 9 finished with value: 0.043677475303411484 and parameters: {'learning_rate': 1.58499239758813e-05, 'batch_size': 17, 'num_train_epochs': 3}. Best is trial 0 with value: 0.014734666794538498.


Best hyperparameters: {'learning_rate': 1.0696001869777201e-05, 'batch_size': 22, 'num_train_epochs': 4}


In [27]:
# 최적화된 하이퍼파라미터 출력
print(f"Best hyperparameters: {study.best_trial.params}")

Best hyperparameters: {'learning_rate': 1.0696001869777201e-05, 'batch_size': 22, 'num_train_epochs': 4}


In [28]:
eval_results = trainer.evaluate()
print(eval_results)


#{'eval_loss': 0.020677322521805763,
#  'eval_runtime': 30.2996,
#  'eval_samples_per_second': 227.495,
#  'eval_steps_per_second': 28.449,
#  'epoch': 3.0}

100%|██████████| 862/862 [00:39<00:00, 21.78it/s]

{'eval_loss': 0.043677475303411484, 'eval_runtime': 39.0992, 'eval_samples_per_second': 176.295, 'eval_steps_per_second': 22.046, 'epoch': 3.0}


In [18]:
# Fine-tuning된 모델 저장
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

100%|██████████| 862/862 [00:35<00:00, 24.31it/s]


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.txt',
 './fine_tuned_model\\added_tokens.json',
 './fine_tuned_model\\tokenizer.json')

In [25]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# fine-tuning된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")


In [26]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [29]:
questions = df['question'].tolist()
answers = df['answer'].tolist()

def get_embeddings(questions, tokenizer, model):
    model.eval()  # 평가 모드로 전환
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # 입력 토큰화
            inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

            # 출력 얻기
            outputs = model(**inputs, output_hidden_states=True)  # hidden_states를 반환하도록 설정
            
            # BERT의 마지막 레이어 hidden_states에서 [CLS] 토큰의 임베딩을 사용
            cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [batch_size, hidden_size]
            
            # numpy 배열로 변환하여 임베딩 목록에 추가
            embeddings.append(cls_embedding.squeeze().numpy())

    return np.array(embeddings)  # 모든 임베딩을 NumPy 배열로 반환


# fine-tuning 후 새로운 임베딩 생성
question_embeddings = get_embeddings(questions, tokenizer, model)

# 새로운 임베딩 저장
torch.save(question_embeddings, 'new_question_embeddings.pth')


In [41]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('new_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


c:\Temp\ipykernel_12040\1734250886.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('new_question_embeddings.pth')


In [42]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 0
최고 유사도 답변: 정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
코사인 유사도: 1.0


In [43]:
# 예시 질문
input_question = "정수기는 위치가 어디죠?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 0
최고 유사도 답변: 제가 안내해 드리겠습니다. 저를 따라 이동해 주시기 바랍니다.
코사인 유사도: 0.9999990463256836
